**Importing Libraries**

In [52]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np

**Loading dataset**

In [53]:
imdb_reviews = pd.read_csv("IMDB_Dataset.csv")
test_reviews = pd.read_csv("IMDB_Test.csv")

In [54]:
imdb_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


**word Index**

In [55]:
word_index = pd.read_csv("word_index.csv")

In [56]:
word_index.head(n=10)

,word,index
0,the,4
1,a,5
2,and,6
3,of,7
4,to,8
5,is,9
6,in,10
7,i,11
8,this,12
9,that,13


In [57]:
word_index = dict(zip(word_index.word,word_index.index))

In [58]:
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

In [59]:
def review_encoder(text):
    return [word_index.get(word.lower(), word_index.get("<OOV>")) for word in text] 

**Review & Sentiment Split**

In [60]:
train_data,train_labels = imdb_reviews['review'],imdb_reviews['sentiment']
test_data,test_labels = test_reviews['review'],test_reviews['sentiment']

In [61]:
train_data = train_data.apply(lambda review:review.split())
test_data = test_data.apply(lambda review:review.split())

In [62]:
def review_encoder_limited(text):
    encoded = [word_index.get(word.lower(), word_index.get("<UNK>")) for word in text]
    return [index if index < 10000 else word_index["<UNK>"] for index in encoded]

train_data = train_data.apply(review_encoder_limited)
test_data = test_data.apply(review_encoder_limited)


In [63]:
def encode_sentiments(sentiment):
    if sentiment == 'positive' :
        return 1
    else :
        return 0
    
train_labels = train_labels.apply(encode_sentiments)
test_labels = test_labels.apply(encode_sentiments)

In [64]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"],padding='post',maxlen=500)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"],padding='post',maxlen=500)

**Model Creation**

In [65]:
model = keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                         keras.layers.GlobalAveragePooling1D(),
                         keras.layers.Dense(16,activation='relu'),
                         keras.layers.Dense(1,activation='sigmoid')])

C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [66]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [67]:
history = model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.5448 - loss: 0.6908 - val_accuracy: 0.5617 - val_loss: 0.6764
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.6433 - loss: 0.6652 - val_accuracy: 0.6703 - val_loss: 0.6267
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.7251 - loss: 0.6060 - val_accuracy: 0.7758 - val_loss: 0.5473
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7783 - loss: 0.5286 - val_accuracy: 0.8127 - val_loss: 0.4805
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.8190 - loss: 0.4614 - val_accuracy: 0.8229 - val_loss: 0.4313
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.8327 - loss: 0.4168 - val_accuracy: 0.8478 - val_loss: 0.3936
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8541 - loss: 0.3785 - val_accuracy: 0.8380 - val_loss: 0.3806
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.8636 - loss: 0.3527 - val_accuracy: 0.8430 - v

In [68]:
loss,accuracy=model.evaluate(test_data,test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8892 - loss: 0.2840


**Testing Model**

In [69]:
index=np.random.randint(1,1000)

In [70]:
user_review=test_reviews.loc[index]
print(user_review)

review       MGM tried pairing up and coming young men with...
sentiment                                             negative
Name: 500, dtype: object


In [71]:
user_review = test_data[index]
user_review = np.array([user_review])
if(model.predict(user_review) > 0.5).astype("int32"):
    print("positive sentiment")
else :
    print("negative sentiment")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
negative sentiment
